<a href="https://colab.research.google.com/github/anas-r-dev/focalnet-tensorflow/blob/main/test_notebook_focalnet_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install focalnet-tensorflow 


import tensorflow as tf
from tensorflow import keras
from focalnet_tensorflow import focalnet_xs_srf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# num_classes = 100
# input_shape = (32, 32, 3)

train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:85%]", "train[85%:]"],
    as_supervised=True,
)

In [3]:
RESIZE_TO = 384
CROP_TO = 224
BATCH_SIZE = 64
STEPS_PER_EPOCH = 10
AUTO = tf.data.AUTOTUNE  # optimise the pipeline performance
NUM_CLASSES = 5  # number of classes
SCHEDULE_LENGTH = (
    500  # we will train on lower resolution images and will still attain good results
)
SCHEDULE_BOUNDARIES = [
    200,
    300,
    400,
]  # more the dataset size the schedule length increase
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
num_classes = 5  # number of classes

In [4]:
@tf.function
def preprocess_train(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.resize(image, (RESIZE_TO, RESIZE_TO))
    image = tf.image.random_crop(image, (CROP_TO, CROP_TO, 3))
    image = image / 255.0
    return (image, label)


@tf.function
def preprocess_test(image, label):
    image = tf.image.resize(image, (RESIZE_TO, RESIZE_TO))
    image = image / 255.0
    return (image, label)


In [5]:
# Training pipeline
pipeline_train = (
    train_ds.shuffle(10000)
    # .repeat(repeat_count)  # Repeat dataset_size / num_steps
    .map(preprocess_train, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

# Validation pipeline
pipeline_validation = (
    validation_ds.map(preprocess_test, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [6]:
model = focalnet_xs_srf(num_classes = num_classes)

In [7]:
# a = tf.random.uniform((1, 224, 224, 3))
b = model(a)

In [12]:
model.summary()

Model: "focal_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 patch_embed (PatchEmbed)    multiple                  2448      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 basic_layer (BasicLayer)    multiple                  78822     
                                                                 
 basic_layer_1 (BasicLayer)  multiple                  230790    
                                                                 
 layer_normalization_10 (Lay  multiple                 192       
 erNormalization)                                                
                                                                 
 adaptive_average_pooling1d   multiple                 0         
 (AdaptiveAveragePooling1D)                              

In [11]:
optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )
model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )
history = model.fit(
    pipeline_train,
    batch_size=BATCH_SIZE,
    epochs=1,
    # steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=pipeline_validation,
    # callbacks=train_callbacks,
)

49/49 [==============================] - 33s 541ms/step - loss: 0.5293 - accuracy: 0.8096 - top-5-accuracy: 1.0000 - val_loss: 0.5476 - val_accuracy: 0.7855 - val_top-5-accuracy: 1.0000
